In [13]:
import cvxopt
# import cvxopt.glpk
from cvxopt import matrix, solvers
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime
import seaborn as sns
import itertools
from scipy.linalg import block_diag
import time
import os
import pickle
from scipy.misc import comb
from itertools import combinations
from functools import partial

In [15]:
from __future__ import print_function

import sys

import cplex
from cplex.exceptions import CplexError

In [170]:
profile = np.array([list(np.random.permutation(range(1,11))) for i in range(5)])

In [179]:
stringmatrix = '['
for i in range(profile.shape[0]):
    for j in range(profile.shape[1]):
        if j!=profile.shape[1]-1:
            stringmatrix+=str(profile[i][j])+','
        else:
            stringmatrix+=str(profile[i][j])+';'
stringmatrix=stringmatrix[:-1]
stringmatrix+=']'
print (stringmatrix)

[2,4,3,1,9,7,8,6,5,10;10,4,1,8,5,9,7,2,3,6;2,3,8,10,7,1,4,9,5,6;10,5,4,7,9,2,1,8,6,3;7,1,4,2,9,6,8,10,3,5]


In [58]:
for p in [1,2,3]:
    for typ in ['rate','sort']:
        with open('./q{}_{}_profile.p'.format(p,typ),'rb') as file_:
            profile = pickle.load(file_).astype(np.int64)
            print(profile)
            n,m = len(profile), len(profile[0])
            # print(n, m)

            k=5

            def Ind(m,k,i,r):
                return (i-1)*(m-k+1)+r+m
            IndY = partial(Ind,m,k)
            # IndY(1,1)

            nIneq  = int(n*((m-k+2)*(m-k+3)/2-1)+comb(m,k))
            nEq = 1
            nVar = int(m+n*(m-k+1)+1)
            Aineq = np.zeros((nIneq, nVar))
            bineq = np.zeros((nIneq, 1))
            Aeq = np.zeros((nEq, nVar))
            beq = np.zeros((nEq, 1))
            f = np.zeros((nVar, 1))
            ctype = 'B'*(nVar-1)+'C'
            f[-1] = 1
            Aeq[0][0:m] = 1
            beq = k
            countIneq = 1

            for i in range(1,n+1):
                for r in range(1,m-k+2):
                    for j in range(1,r+1):
                        Aineq[countIneq-1, IndY(i,r)-1] = 1
            #             print(IndY(i,r))
            #             print(Aineq)
                        Aineq[countIneq-1, profile[i-1,j-1]-1] = 1
            #             print (Aineq)
                        bineq[countIneq-1] = 1
                        countIneq = countIneq + 1
                    Aineq[countIneq-1, IndY(i,r)-1] = -1
                    for j in range(1,r+1):
                        Aineq[countIneq-1, profile[i-1,j-1]-1] = -1
                    bineq[countIneq-1] = -1
                    countIneq = countIneq + 1
            list_T = np.array([list(int(y)+1 for y in x) for x in list(combinations(range(m),k))])
            for indT in range(1, list_T.shape[0]+1):
                T = list_T[indT-1]
                first_indices_T = np.array([[i for i,elem in enumerate(x) if x[i] in T][0] + 1 for x in profile])
                Aineq[countIneq-1, nVar-1] = -1
                for i in range(1,n+1):
                    Aineq[countIneq-1,IndY(i,first_indices_T[i-1])-1] = 1./first_indices_T[i-1]
                countIneq = countIneq + 1

            # data common to all populateby functions
            my_obj = list(i[0] for i in f)
            my_ub = [cplex.infinity]*len(f)
            my_lb = [0.0]*len(f)
            my_ctype = ctype
            my_colnames = ['x{}'.format(i) for i in range(len(f))]
            my_rhs = list(i[0] for i in bineq) + [float(beq)]
            my_rownames = ['r{}'.format(i) for i in range(nIneq+nEq)]
            my_sense = 'L'*nIneq+'E'*nEq
        
            def populatebynonzero(prob):
                prob.objective.set_sense(prob.objective.sense.minimize)

                prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                            names=my_rownames)
                prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype,
                                   names=my_colnames)

                rows, cols = np.nonzero(np.concatenate((Aineq,Aeq)))
                rows=list(rows)
                cols=list(cols)
                vals = [np.concatenate((Aineq,Aeq))[rows[i],cols[i]] for i in range(len(rows))]

                prob.linear_constraints.set_coefficients(zip(rows, cols, vals))


            def mipex1(pop_method):

                try:
                    my_prob = cplex.Cplex()

                    handle = populatebynonzero(my_prob)

                    my_prob.solve()
                except CplexError as exc:
                    print(exc)
                    return

                print()
                # solution.get_status() returns an integer code
                print("Solution status = ", my_prob.solution.get_status(), ":", end=' ')
                # the following line prints the corresponding string
                print(my_prob.solution.status[my_prob.solution.get_status()])
                print("Solution value  = ", my_prob.solution.get_objective_value())

                numcols = my_prob.variables.get_num()
                numrows = my_prob.linear_constraints.get_num()

                slack = my_prob.solution.get_linear_slacks()
                x = my_prob.solution.get_values()

            #     for j in range(numrows):
            #         print("Row %d:  Slack = %10f" % (j, slack[j]))
                for j in range(m):
                    print("Column %d:  Value = %10f" % (j, x[j]))
                print ("Column %d:  Value = %10f" % (numcols-1, x[numcols-1]))
                return x[:10]

            choices = mipex1('n')
            print (list((np.array(np.where(np.array(choices)>0.0))+1)[0]))
            selected_points = list((np.array(np.where(np.array(choices)>0.0))+1)[0])
            with open('./q{}_{}_results.p'.format(p,typ),'wb') as file_:
                pickle.dump(selected_points, file_)

[[ 4  7  9  8  3  2  6  1  5 10]
 [ 8  1  3  7  9  2  5  6 10  4]
 [ 1  4  8  5  2 10  9  7  6  3]
 [ 7  2  6  9  1  5  8  3  4 10]
 [ 8  1  5  4  9 10  6  3  2  7]
 [ 2  9  3  7  1 10  6  5  4  8]
 [10  8  5  2  4  3  9  7  1  6]
 [ 1  6  4 10  8  9  7  5  3  2]
 [ 9 10  8  1  4  5  3  2  7  6]
 [ 1 10  4  2  7  3  6  9  8  5]
 [ 2  8  4  3  7  9  5 10  6  1]
 [ 9  5  3  8 10  2  7  1  6  4]
 [ 3  2  9  1  4  7  6  8 10  5]
 [ 3  2 10  6  9  4  5  7  1  8]
 [ 5  9  7  1  8  6 10  2  3  4]
 [ 1  9  2  7  6  8  5  3  4 10]]
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0


Tried aggregator 2 times.
MIP Presolve eliminated 16 rows and 0 columns.
Aggregator did 16 substitutions.
Reduced MIP has 653 rows, 91 columns, and 4452 nonzeros.
Reduced MIP has 90 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.11 ticks)
Found incumbent of value 13.833333 after 0.03 sec. (8.77 ticks)
Probing fixed 16 vars, tightened 0 bounds.
Probing time = 0.02 sec. (11.10 ticks)
Cover probing fixed 0 vars, tightened 1 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 138 rows and 22 columns.
Reduced MIP has 515 rows, 69 columns, and 3912 nonzeros.
Reduced MIP has 68 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.23 ticks)
Probing time = 0.01 sec. (3.29 ticks)
Tried aggregator 1 time.
Reduced MIP has 515 rows, 69 columns, and 3912 nonzeros.
Reduced MIP has 68 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.49 ticks)
Probing time = 0.01 sec. (3.27 ticks)
Clique table members: 1103.
MIP emph

Tried aggregator 2 times.
MIP Presolve eliminated 67 rows and 0 columns.
Aggregator did 17 substitutions.
Reduced MIP has 628 rows, 96 columns, and 3746 nonzeros.
Reduced MIP has 95 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.74 ticks)
Found incumbent of value 6.000000 after 0.02 sec. (5.32 ticks)
Probing fixed 17 vars, tightened 0 bounds.
Probing time = 0.01 sec. (8.90 ticks)
Cover probing fixed 0 vars, tightened 8 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 229 rows and 40 columns.
Reduced MIP has 399 rows, 56 columns, and 2704 nonzeros.
Reduced MIP has 55 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.57 ticks)
Probing fixed 18 vars, tightened 0 bounds.
Probing time = 0.01 sec. (3.44 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 121 rows and 19 columns.
Reduced MIP has 278 rows, 37 columns, and 1943 nonzeros.
Reduced MIP has 36 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 se

Tried aggregator 2 times.
MIP Presolve eliminated 14 rows and 0 columns.
Aggregator did 10 substitutions.
Reduced MIP has 499 rows, 61 columns, and 3142 nonzeros.
Reduced MIP has 60 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.13 ticks)
Found incumbent of value 3.000000 after 0.02 sec. (3.59 ticks)
Probing fixed 10 vars, tightened 0 bounds.
Probing time = 0.01 sec. (5.41 ticks)
Cover probing fixed 0 vars, tightened 7 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 114 rows and 17 columns.
Reduced MIP has 385 rows, 44 columns, and 2746 nonzeros.
Reduced MIP has 43 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.91 ticks)
Probing fixed 23 vars, tightened 0 bounds.
Probing time = 0.00 sec. (1.85 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 195 rows and 23 columns.
MIP Presolve modified 16 coefficients.
Reduced MIP has 190 rows, 21 columns, and 1249 nonzeros.
Reduced MIP has 20 binaries, 0 generals, 0 SOSs, an

Tried aggregator 2 times.
MIP Presolve eliminated 48 rows and 0 columns.
Aggregator did 13 substitutions.
Reduced MIP has 543 rows, 76 columns, and 3247 nonzeros.
Reduced MIP has 75 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.72 ticks)
Found incumbent of value 12.000000 after 0.02 sec. (6.44 ticks)
Probing fixed 13 vars, tightened 0 bounds.
Probing time = 0.01 sec. (6.25 ticks)
Cover probing fixed 0 vars, tightened 2 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 115 rows and 18 columns.
Reduced MIP has 428 rows, 58 columns, and 2732 nonzeros.
Reduced MIP has 57 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.69 ticks)
Probing time = 0.00 sec. (2.52 ticks)
Tried aggregator 1 time.
Reduced MIP has 428 rows, 58 columns, and 2732 nonzeros.
Reduced MIP has 57 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.71 ticks)
Probing time = 0.00 sec. (2.52 ticks)
Clique table members: 778.
MIP emphas

Tried aggregator 2 times.
MIP Presolve eliminated 81 rows and 0 columns.
Aggregator did 15 substitutions.
Reduced MIP has 562 rows, 86 columns, and 3160 nonzeros.
Reduced MIP has 85 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (3.95 ticks)
Found incumbent of value 3.000000 after 0.02 sec. (4.44 ticks)
Probing fixed 15 vars, tightened 0 bounds.
Probing time = 0.01 sec. (6.60 ticks)
Cover probing fixed 0 vars, tightened 31 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 498 rows and 69 columns.
MIP Presolve modified 75 coefficients.
Reduced MIP has 64 rows, 17 columns, and 283 nonzeros.
Reduced MIP has 16 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.64 ticks)
Probing fixed 2 vars, tightened 0 bounds.
Probing changed sense of 3 constraints.
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 40 rows and 5 columns.
MIP Presolve modified 40 coefficients.
Reduced MIP has 24 rows, 12 colum

Tried aggregator 2 times.
MIP Presolve eliminated 23 rows and 0 columns.
Aggregator did 11 substitutions.
Reduced MIP has 516 rows, 66 columns, and 3256 nonzeros.
Reduced MIP has 65 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.36 ticks)
Found incumbent of value 9.500000 after 0.02 sec. (5.66 ticks)
Probing fixed 11 vars, tightened 0 bounds.
Probing time = 0.01 sec. (5.80 ticks)
Cover probing fixed 0 vars, tightened 1 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 84 rows and 13 columns.
Reduced MIP has 432 rows, 53 columns, and 3019 nonzeros.
Reduced MIP has 52 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.79 ticks)
Probing time = 0.00 sec. (2.60 ticks)
Tried aggregator 1 time.
Reduced MIP has 432 rows, 53 columns, and 3019 nonzeros.
Reduced MIP has 52 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.84 ticks)
Probing time = 0.00 sec. (2.59 ticks)
Clique table members: 651.
MIP emphasis

In [35]:
selected_points = list((np.array(np.where(np.array(choices)>0.0))+1)[0])
selected_points

[1, 2, 3, 8, 9]

In [197]:
matlab = np.loadtxt('../nisarg_code/testmatrix', delimiter=',')

In [198]:
print(matlab)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]]


In [226]:
print(Aineq)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]]


In [49]:
with open('./q2_sort_profile.p','rb') as file_:
    profile = pickle.load(file_)
profile.astype(np.int64)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10,  0,  0,  0,  0,  0, 10, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10,  0,  0, 10, 10,  0,  0, 10,  0],
       [10,  0,  0,  0, 10,  0,  0, 10, 10, 10],
       [ 0,  0,  0,  0, 10, 10, 10, 10, 10,  0],
       [ 0,  0,  0,  0, 10, 10, 10, 10, 10,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10,  0,  0,  0, 10,  0, 10,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [56]:
p

NameError: name 'p' is not defined